In [ ]:
# https://github.com/nikitaa30/Content-based-Recommender-System/blob/master/recommender_system.py
# https://heartbeat.fritz.ai/recommender-systems-with-python-part-i-content-based-filtering-5df4940bd831

## Building NLP Content-based RS
##### https://medium.com/@armandj.olivares/building-nlp-content-based-recommender-systems-b104a709c042
##### https://github.com/ArmandDS/jobs_recommendations/blob/master/job_analysis_content_recommendation.ipynb


### Clean Text

In [14]:
import pandas as pd
import numpy as np
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/reginaduval/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/reginaduval/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/reginaduval/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/reginaduval/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
# already cleaned and saved as reviews10k_cleaned2.csv for review_text or reviews10k_grouped_full for full_text

In [15]:
ds = pd.read_csv('Data/reviews10k_grouped_full.csv')

In [11]:
ds.head()

,Unnamed: 0,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,original_publication_year,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,authors_y,original_title_y,review_text,full_text
0,0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,2008.0,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,Suzanne Collins,The Hunger Games,I cracked and finally picked this up. Very enj...,"2008.0 The Hunger Games (The Hunger Games, #1)..."
1,1,2,3,3,4640799,491,439554934,9.780440e+12,1997.0,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,"J.K. Rowling, Mary GrandPré",Harry Potter and the Philosopher's Stone,Tuve el gusto de leerlo antes de que saliera l...,1997.0 Harry Potter and the Sorcerer's Stone (...
2,2,3,41865,41865,3212258,226,316015849,9.780316e+12,2005.0,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,Stephenie Meyer,Twilight,"If I was 15, I would have LOVED this. It wasn'...","2005.0 Twilight (Twilight, #1) en-US Stephenie..."
3,3,4,2657,2657,3275794,487,61120081,9.780061e+12,1960.0,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,Harper Lee,To Kill a Mockingbird,"Still relevant and beautiful.,R.I.P. Ms. Harpe...",1960.0 To Kill a Mockingbird eng Harper Lee To...
4,4,5,4671,4671,245494,1356,743273567,9.780743e+12,1925.0,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,F. Scott Fitzgerald,The Great Gatsby,I read this classic twice in high school. Its ...,1925.0 The Great Gatsby eng F. Scott Fitzgeral...


### Extract features from text

In [16]:
pd.set_option('display.max_columns', 100)
#ds = pd.read_csv('Data/reviews10k_cleaned2.csv')
ds.head()

,Unnamed: 0,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,original_publication_year,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,authors_y,original_title_y,review_text,full_text
0,0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,2008.0,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,Suzanne Collins,The Hunger Games,I cracked and finally picked this up. Very enj...,"2008.0 The Hunger Games (The Hunger Games, #1)..."
1,1,2,3,3,4640799,491,439554934,9.780440e+12,1997.0,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,"J.K. Rowling, Mary GrandPré",Harry Potter and the Philosopher's Stone,Tuve el gusto de leerlo antes de que saliera l...,1997.0 Harry Potter and the Sorcerer's Stone (...
2,2,3,41865,41865,3212258,226,316015849,9.780316e+12,2005.0,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,Stephenie Meyer,Twilight,"If I was 15, I would have LOVED this. It wasn'...","2005.0 Twilight (Twilight, #1) en-US Stephenie..."
3,3,4,2657,2657,3275794,487,61120081,9.780061e+12,1960.0,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,Harper Lee,To Kill a Mockingbird,"Still relevant and beautiful.,R.I.P. Ms. Harpe...",1960.0 To Kill a Mockingbird eng Harper Lee To...
4,4,5,4671,4671,245494,1356,743273567,9.780743e+12,1925.0,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,F. Scott Fitzgerald,The Great Gatsby,I read this classic twice in high school. Its ...,1925.0 The Great Gatsby eng F. Scott Fitzgeral...


In [17]:
# for tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf_rev = tfidf_vectorizer.fit_transform((ds['full_text']))
tfidf_rev

<10000x1299420 sparse matrix of type '<class 'numpy.float64'>'
	with 41214251 stored elements in Compressed Sparse Row format>

In [18]:
# for count
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

count_rev = count_vectorizer.fit_transform((ds['full_text']))
count_rev

<10000x1299420 sparse matrix of type '<class 'numpy.int64'>'
	with 41214251 stored elements in Compressed Sparse Row format>

### Evaluating the recommendations

In [19]:
#insert goodreads_book_id
g = 1914973
index = np.where(ds['goodreads_book_id'] == g)[0][0]
read_book = ds.iloc[[index]]
read_book
#read_book = ds.loc[ds['original_title'] == g]
#read_book

,Unnamed: 0,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,original_publication_year,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,authors_y,original_title_y,review_text,full_text
1324,1324,1325,1914973,1914973,6504351,56,670018864,9.780670e+12,2008.0,"The Likeness (Dublin Murder Squad, #2)",eng,4.05,72480,80243,6945,915,2856,14814,34478,27180,Tana French,The Likeness,Word on the street was that this book wasn't a...,"2008.0 The Likeness (Dublin Murder Squad, #2) ..."


### Content based RS with tfidf

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
book_tfidf = tfidf_vectorizer.transform(read_book['full_text'])
cos_similarity_tfidf = map(lambda x: cosine_similarity(book_tfidf, x), tfidf_rev)
output = list(cos_similarity_tfidf)

### Content based RS with CountVectorizer

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
book_count = count_vectorizer.transform(read_book['full_text'])
cos_similarity_countv = map(lambda x: cosine_similarity(book_count, x), count_rev)
output2 = list(cos_similarity_countv)

### Top recommendations

In [22]:
def get_recommendation(top, ds, scores):
  recommendation = pd.DataFrame(columns = ['goodreads_book_id', 'authors', 'title', 'score'])
  count = 0
  for i in top:
      recommendation.at[count, 'goodreads_book_id'] = ds.iloc[i, 2]
      recommendation.at[count, 'authors'] = ds.iloc[i, 20]
      recommendation.at[count, 'title'] = ds.iloc[i, 9]
      recommendation.at[count, 'score'] =  scores[count]
      count += 1
  return recommendation

In [23]:
# for tfidf
top = sorted(range(len(output)), key=lambda i: output[i], reverse=True)[:10]
list_scores = [output[i][0][0] for i in top]
get_recommendation(top, ds, list_scores)

,goodreads_book_id,authors,title,score
0,1914973,Tana French,"The Likeness (Dublin Murder Squad, #2)",1
1,237209,Tana French,"In the Woods (Dublin Murder Squad, #1)",0.894628
2,67513,Karen Chance,"Touch the Dark (Cassandra Palmer, #1)",0.874744
3,3665811,L.J. Smith,The Captive Part II / The Power (The Secret Ci...,0.870654
4,16101128,Rick Yancey,"The 5th Wave (The 5th Wave, #1)",0.868763
5,3099780,L.J. Smith,The Initiation / The Captive Part I (The Secre...,0.867556
6,13597723,Jennifer Lynn Barnes,"The Naturals (The Naturals, #1)",0.855328
7,310459,Mildred D. Taylor,"Roll of Thunder, Hear My Cry (Logans, #4)",0.852976
8,16131489,Rick Yancey,"The Last Star (The 5th Wave, #3)",0.845764
9,7093952,Tana French,"Faithful Place (Dublin Murder Squad, #3)",0.834554


In [24]:
# for count
top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:10]
list_scores = [output2[i][0][0] for i in top]
get_recommendation(top, ds, list_scores)

,goodreads_book_id,authors,title,score
0,1914973,Tana French,"The Likeness (Dublin Murder Squad, #2)",1
1,237209,Tana French,"In the Woods (Dublin Murder Squad, #1)",0.97796
2,10805160,Tana French,"Broken Harbour (Dublin Murder Squad, #4)",0.977658
3,29430013,Tana French,The Trespasser (Dublin Murder Squad #6),0.976405
4,67513,Karen Chance,"Touch the Dark (Cassandra Palmer, #1)",0.974768
5,7093952,Tana French,"Faithful Place (Dublin Murder Squad, #3)",0.972591
6,20821043,Tana French,"The Secret Place (Dublin Murder Squad, #5)",0.972075
7,7058405,"Camilla Läckberg, Steven T. Murray","The Ice Princess (Patrik Hedström, #1)",0.971535
8,10836728,Daniel O'Malley,"The Rook (The Checquy Files, #1)",0.971449
9,207365,Karen Chance,"Claimed By Shadow (Cassandra Palmer, #2)",0.971332


In [25]:
# build a wordcloud

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import itertools

In [26]:
review_text = itertools.islice(ds.full_text, 5000)
review_text = list(review_text)
review_text

In [ ]:
#review_text = itertools.islice(ds.review_text, 10000)
#review_text = list(review_text)
bunch_text = " ".join(str(text) for text in review_text[:1500])
stopwords = set(STOPWORDS)
wordcloud = WordCloud(stopwords=stopwords, background_color="white", colormap= "magma").generate(bunch_text)

In [ ]:
plt.figure(figsize=[20,20])
plt.imshow(wordcloud, interpolation="sinc")
plt.axis("off")
plt.show()